# Importing

In [2]:
import datetime

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

import pandas as pd
from pandas_datareader import data

from scipy.signal import savgol_filter
import statsmodels
import statsmodels.api as sm
import scipy.stats as stats
import scipy.optimize as optimize
import numpy as np
#from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
#For the usual plots!
#matplotlib inline
#Makes graphs in jupyter interactive!
%matplotlib widget

In [3]:
import warnings
#Only showing a warning once!
warnings.filterwarnings(action='once')

#Removing warnings from displaying!
warnings.filterwarnings('ignore')

In [4]:
plt.close('all')

def smoothing_sma_savgol(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        savgol_len = len(dfsma[(SMA + '_change', t)].dropna())
        if (savgol_len % 2) == 0:
            savgol_len += 1
            if (savgol_len % 2) != 0:
                return savgol_len
            else:
                continue
    #No real rules for polyorder, just use what fits best for its purpose!
        savgol = savgol_filter(dfsma[(SMA + '_change', t)].dropna(), savgol_len, polyorder=100)
        insert_len =  len(dfsma[(SMA + '_change', t)]) - len(savgol)
        savgol = np.insert(savgol, 0, np.zeros(insert_len) + np.nan)
        savgol = pd.Series(savgol, index=dfsma.index)
        dfsma[(SMA + '_change_smooth_savgol', t)] = savgol
 #BETERE Variant van de fucntie::::!!!       
#def smoothing_sma_savgol(tickers, df):
#    dftemp = df.copy()
#    for t in tickers:
#        savgol_len = len(df[('SMA_change', t)].dropna())
#        while (savgol_len % 2) == 0:
#            savgol_len -= 1
#    #No real rules for polyorder, just use what fits best for its purpose!
#        savgol = savgol_filter(df[('SMA_change', t)].dropna(), savgol_len, polyorder=100)
#        insert_len =  len(df[('SMA_change', t)]) - len(savgol)
#        savgol = np.insert(savgol, 0, np.zeros(insert_len) + np.nan)
#        dftemp[('Savgol', t)] = pd.Series(savgol, index=df.index)
#        dftemp[('Savgol', t)] = dftemp[('Savgol', t)]
#    dftemp.reset_index()
#    return dftemp

def polyfit_df(tickers):
    dftemp = df.copy()
    for t in tickers:
        dftemp[('SMA20_change', t)] = dfsma[('SMA20_change', t)].dropna()
        result = np.polyfit(dfsma[('SMA20_change', t)].dropna().index, dfsma[('SMA20_change', t)].dropna(), 50)
        poly_eq = np.poly1d(result)
        dftemp[('Polyfit', t)] = poly_eq(dftemp.index)
        #dftemp[('Polyfit_change', t)] = dftemp[('Polyfit', t)].pct_change()  <--????????
    dftemp = dftemp.reset_index()
    return dftemp
        
def smoothing_sma_sma(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_sma', t)] = dfsma[(SMA + '_change', t)].rolling(x).mean()
        
def smoothing_sma_ema(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_ema', t)] = dfsma[(SMA + '_change', t)].ewm(span=x, adjust=False).mean()
        
def smoothing_sma_smoothed(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_smoothed', t)] = dfsma[(SMA + '_change_smooth_sma', t)].rolling(x).mean()
    
def smoothing_sma_ar(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_ar', t)] = np.polyfit(dfsma.index.values, dfsma[(SMA + '_change', t)], 4)

# Data

In [5]:
tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'KODK', 'INTC', 'PANW']
startd = datetime.datetime(2016, 1, 1) 
endd = datetime.datetime(2020, 1, 1)

df = data.DataReader(tickers, 'yahoo', startd, endd).reset_index()
#df.set_index('Date', inplace=True)            <-- weggehaald zodat polyfit en backtest "i-1" werken

#uitrekenen van de daily return (cert)
#oude manieren:
#df[('Close1', 'AAPL')] = df[('Close', 'AAPL')].shift(1)   <- als je het los wilt doen
#df[('Return', 'AAPL')] = df[('Close', 'AAPL')].shift(1) - df[('Close', 'AAPL')]
#df[('Return', 'MSFT')] = df[('Close', 'MSFT')].shift(1) - df[('Close', 'MSFT')]
#huidige manier:
for ticker in tickers:
    df[('Return', ticker)] = df[('Close', ticker)].shift(1) - df[('Close', ticker)]
    
for ticker in tickers:
    df[('Percent_return', ticker)] = df[('Close', ticker)].pct_change()
    
df.tail()

#Andere data kan ook via data.DataReader (OECD, Eurostat,...): https://pandas-datareader.readthedocs.io/en/latest/remote_data.html
#Stack overflow voor yahoo: https://stackoverflow.com/questions/49604224/pulling-stock-information-using-pandas-datareader

Attributes       Date   Adj Close                                     \
Symbols                       SPY       AAPL        MSFT        PYPL   
1001       2019-12-24  316.685272  69.886818  155.730255  108.690002   
1002       2019-12-26  318.371063  71.273392  157.006729  109.750000   
1003       2019-12-27  318.292175  71.246353  157.293686  109.400002   
1004       2019-12-30  316.537384  71.669212  155.938049  107.970001   
1005       2019-12-31  317.306335  72.192863  156.046890  108.169998   

Attributes                                                      ...    Return  \
Symbols           ISRG         GM  KODK       INTC        PANW  ...      PANW   
1001        598.809998  36.139141  3.55  57.968597  231.789993  ... -0.459991   
1002        598.039978  36.040344  3.54  58.368649  233.539993  ... -1.750000   
1003        597.880005  36.119381  3.83  58.622341  233.270004  ...  0.269989   
1004        591.320007  36.000824  4.16  58.173500  231.429993  ...  1.840012   
1005        591.150024  36.158894  4.65  58.397919  231.250000  ...  0.179993   

Attributes Percent_return                                                    \
Symbols               SPY      AAPL      MSFT      PYPL      ISRG        GM   
1001             0.000031  0.000951 -0.000191  0.000737  0.002327 -0.005167   
1002             0.005323  0.019840  0.008197  0.009752 -0.001286 -0.002734   
1003            -0.000248 -0.000379  0.001828 -0.003189 -0.000267  0.002193   
1004            -0.005513  0.005935 -0.008619 -0.013071 -0.010972 -0.003282   
1005             0.002429  0.007307  0.000698  0.001852 -0.000287  0.004391   

Attributes                                
Symbols         KODK      INTC      PANW  
1001       -0.008380  0.003039  0.001988  
1002       -0.002817  0.006901  0.007550  
1003        0.081921  0.004346 -0.001156  
1004        0.086162 -0.007657 -0.007888  
1005        0.117789  0.003858 -0.000778  

[5 rows x 73 columns]

# Beautiful Soup; scraping price targets!

In [6]:
##Basics van programming with data Module 11
##Iets meer van ->  https://hackersandslackers.com/scraping-urls-with-beautifulsoup/

#Marketwatch getting price target <- minst goeie website, maar wel het makkelijkste

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

url = 'https://www.marketwatch.com/investing/stock/msft/analystestimates'
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())
# documentation:  https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all
#print(soup.find_all('td')) 
# .find() is genoeg, want het is de 1e waarde die ik moet hebben!
print(soup.find('td', class_='').text)

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
##Basics van programming with data Module 11
##Iets meer van ->  https://hackersandslackers.com/scraping-urls-with-beautifulsoup/

#CNN getting price targets <- geeft alle price targets!

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

url = f'https://money.cnn.com/quote/forecast/forecast.html?symb={tickers[1]}'
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())

# documentation:  https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all
ps = soup.find_all('p')
# Algemene documentatie, dus ook dat ja kan slicen:  https://www.crummy.com/software/BeautifulSoup/bs3/documentation.html
extract = ps[1:3]
goal = ''
for string in extract:
    goal += string.text
goal = goal.rstrip('Move your mouse over pastmonths for detail')
print(goal)

# Ichimoku cloud

In [7]:
##need to add the .copy() at the end when creating a sub-df. -> https://stackoverflow.com/questions/34682828/extracting-specific-selected-columns-to-new-dataframe-as-a-copy
##.dropna() because holidays are included in the data!
#dfichi = df[[('Date', ''), ('High', 'AAPL'), ('Low', 'AAPL'), ('Close', 'AAPL')]].copy().dropna()
dfichi = df.copy().dropna() #werkt voor alle tickers!  -> Probleem verwijdert data voor alles als data ontbreekt
#dfichi.set_index('Date', inplace=True)

##this gives a table with ALL values -> https://stackoverflow.com/questions/19124601/pretty-print-an-entire-pandas-series-dataframe
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    display(dfichi)
for t in tickers:
# Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
    nine_period_high = dfichi[('High', t)].rolling(window= 9).max()
    nine_period_low = dfichi[('Low', t)].rolling(9).min()
    dfichi[('Tenkan_sen', t)] = (nine_period_high + nine_period_low) /2

# Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    period26_high = dfichi[('High', t)].rolling(26).max()
    period26_low = dfichi[('Low', t)].rolling(26).min()
    dfichi[('Kijun_sen', t)] = (period26_high + period26_low) / 2

# Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
    dfichi[('Senkou_span_a', t)] = ((dfichi[('Tenkan_sen', t)] + dfichi[('Kijun_sen', t)]) / 2).shift(26)

# Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
    period52_high = dfichi[('High', t)].rolling(52).max()
    period52_low = dfichi[('Low', t)].rolling(52).min()
    dfichi[('Senkou_span_b', t)] = ((period52_high + period52_low) / 2).shift(26)

# The most current closing price plotted 26 time periods behind (optional)
    dfichi[('Chikou_span', t)] = dfichi[('Close', t)].shift(-26)


##SOURCE: https://stackoverflow.com/questions/28477222/python-pandas-calculate-ichimoku-chart-components
# this is to extend the 'df' in future for 26 days
# the 'df' here is numerical indexed df
#last_index = dfichi.iloc[-1:].index[0]
#last_date = dfichi[('Date', '')].iloc[-1].date()
#for i in range(26):
#    dfichi.loc[last_index+1 +i, 'Date'] = last_date + timedelta(days=i)

##Source: re-index the df with the extra dates! -> https://stackoverflow.com/questions/19119039/pandas-extend-index-of-a-dataframe-setting-all-columns-for-new-rows-to-nan
##Adding dates and times in python:  http://www.pressthered.com/adding_dates_and_times_in_python/
#dfichi.index += datetime.timedelta(days=26)


# create a quick plot of the results to see what we have created
display(dfichi)
#print(dfichi.index.values) gives all index values!
#this part is from: https://blog.quantinsti.com/ichimoku-cloud-trading-strategy/
fig, ax = plt.subplots(figsize=(10,4))
ax.plot(dfichi.index, dfichi[('Tenkan_sen', 'PANW')], color = 'violet')
ax.plot(dfichi.index, dfichi[('Kijun_sen', 'PANW')], color = 'r')
ax.plot(dfichi.index, dfichi[('Senkou_span_a', 'PANW')], color = 'lightgreen')
ax.plot(dfichi.index, dfichi[('Senkou_span_b', 'PANW')], color = 'lightcoral')
ax.plot(dfichi.index, dfichi[('Chikou_span', 'PANW')], color = 'green')
ax.plot(dfichi.index, dfichi[('Close', 'PANW')], color = 'black')
ax.fill_between(dfichi.index, dfichi[('Senkou_span_a', 'PANW')], dfichi[('Senkou_span_b', 'PANW')], where = dfichi[('Senkou_span_a', 'AAPL')] >= dfichi[('Senkou_span_b', 'AAPL')], color = 'lightgreen')
ax.fill_between(dfichi.index, dfichi[('Senkou_span_a', 'PANW')], dfichi[('Senkou_span_b', 'PANW')], where = dfichi[('Senkou_span_a', 'AAPL')] < dfichi[('Senkou_span_b', 'AAPL')], color = 'lightcoral')
plt.grid()
plt.show()

##Oude manieren van plotten!
#dfichi.plot(figsize=(15,8))
##werkt nog niet
#dfichi.plot(('Date', ''), [('Close', 'AAPL'), ('tenkan_sen', 'AAPL'), ('kijun_sen', 'AAPL'), ('senkou_span_a', 'AAPL'), ('senkou_span_b', 'AAPL'), ('chikou_span', 'AAPL')], figsize=(15,8))


Attributes       Date   Adj Close                                     \
Symbols                       SPY       AAPL        MSFT        PYPL   
1          2016-01-05  183.601654  23.596279   50.234879   34.310001   
2          2016-01-06  181.285645  23.134508   49.322346   33.980000   
3          2016-01-07  176.936371  22.158121   47.606789   33.130001   
4          2016-01-08  174.994186  22.275295   47.752792   32.689999   
5          2016-01-11  175.167450  22.635979   47.725410   33.040001   
...               ...         ...        ...         ...         ...   
1001       2019-12-24  316.685272  69.886818  155.730255  108.690002   
1002       2019-12-26  318.371063  71.273392  157.006729  109.750000   
1003       2019-12-27  318.292175  71.246353  157.293686  109.400002   
1004       2019-12-30  316.537384  71.669212  155.938049  107.970001   
1005       2019-12-31  317.306335  72.192863  156.046890  108.169998   

Attributes                                                       ...  \
Symbols           ISRG         GM   KODK       INTC        PANW  ...   
1           183.986664  26.766094  11.79  29.574759  170.559998  ...   
2           184.413330  25.808695  11.36  28.919096  167.309998  ...   
3           178.966660  24.752245  10.99  27.835068  164.080002  ...   
4           178.846664  24.372585  10.47  27.546576  162.740005  ...   
5           179.860001  24.966835   9.76  28.027397  162.020004  ...   
...                ...        ...    ...        ...         ...  ...   
1001        598.809998  36.139141   3.55  57.968597  231.789993  ...   
1002        598.039978  36.040344   3.54  58.368649  233.539993  ...   
1003        597.880005  36.119381   3.83  58.622341  233.270004  ...   
1004        591.320007  36.000824   4.16  58.173500  231.429993  ...   
1005        591.150024  36.158894   4.65  58.397919  231.250000  ...   

Attributes Tenkan_sen  Kijun_sen Senkou_span_a Senkou_span_b Chikou_span  \
Symbols          INTC       INTC          INTC          INTC        INTC   
1                 NaN        NaN           NaN           NaN   28.219999   
2                 NaN        NaN           NaN           NaN   28.639999   
3                 NaN        NaN           NaN           NaN   28.780001   
4                 NaN        NaN           NaN           NaN   29.469999   
5                 NaN        NaN           NaN           NaN   29.420000   
...               ...        ...           ...           ...         ...   
1001        58.195000  57.764999     56.402499     53.664999         NaN   
1002        58.394999  57.799999     56.402499     53.664999         NaN   
1003        58.709999  58.115000     56.485000     53.664999         NaN   
1004        58.709999  58.115000     56.485000     53.664999         NaN   
1005        58.709999  58.115000     56.494999     53.664999         NaN   

Attributes  Tenkan_sen   Kijun_sen Senkou_span_a Senkou_span_b Chikou_span  
Symbols           PANW        PANW          PANW          PANW        PANW  
1                  NaN         NaN           NaN           NaN  124.370003  
2                  NaN         NaN           NaN           NaN  123.739998  
3                  NaN         NaN           NaN           NaN  127.739998  
4                  NaN         NaN           NaN           NaN  132.669998  
5                  NaN         NaN           NaN           NaN  123.059998  
...                ...         ...           ...           ...         ...  
1001        228.690002  232.924995    233.820000    224.500000         NaN  
1002        230.344994  232.924995    234.769997    225.334999         NaN  
1003        230.594994  232.924995    235.040001    225.510002         NaN  
1004        231.419998  232.924995    235.325005    225.795006         NaN  
1005        231.449997  232.924995    235.907505    225.795006         NaN  

[1005 rows x 118 columns]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Moving Averages

In [286]:
#Source: https://www.learndatasci.com/tutorials/python-finance-part-3-moving-average-trading-strategy/

#Groot probleem opgelost!!! misschien?
#Probleem was: op iedere plek was wel een "NaN", dus wanneer ik "dropna()" gebruikt was de df leeg
#opgelost door "dropna()" te doen voordat ik de SMA's berekende, zodate de lege waardes weg zijn en dus geen gaten kaas!
dfsma = df.copy().dropna()    #-> Probleem verwijdert data voor alles als data ontbreekt (bijv door adyen)!!
#dfsma.set_index('Date', inplace=True)
for t in tickers:
    dfsma[('SMA10', t)] = dfsma[('Close', t)].rolling(10).mean()
    dfsma[('SMA20', t)] = dfsma[('Close', t)].rolling(20).mean()
    dfsma[('SMA50', t)] = dfsma[('Close', t)].rolling(50).mean()
    dfsma[('SMA100', t)] = dfsma[('Close', t)].rolling(100).mean()
    dfsma[('SMA200', t)] = dfsma[('Close', t)].rolling(200).mean()
    dfsma[('SMA10_change', t)] = dfsma[('SMA20', t)].pct_change()
    dfsma[('SMA20_change', t)] = dfsma[('SMA20', t)].pct_change()
    dfsma[('SMA50_change', t)] = dfsma[('SMA50', t)].pct_change()
    dfsma[('SMA100_change', t)] = dfsma[('SMA100', t)].pct_change()
    dfsma[('SMA200_change', t)] = dfsma[('SMA200', t)].pct_change()

#display(dfsma)    #om de een of andere reden verwijderd hij alles als ik "dropna()" gebruik, daarvoor is het prima

#The fuction: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html
#Explanantion: https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way
SMA = ['SMA10', 'SMA20', 'SMA50', 'SMA100', 'SMA200']
smoothing_sma_savgol(SMA[0], tickers) 
smoothing_sma_savgol(SMA[1], tickers)
smoothing_sma_savgol(SMA[2], tickers)
smoothing_sma_savgol(SMA[3], tickers)
smoothing_sma_savgol(SMA[4], tickers)
smoothing_sma_sma(SMA[0], tickers)
smoothing_sma_sma(SMA[1], tickers)
smoothing_sma_sma(SMA[2], tickers)
smoothing_sma_sma(SMA[3], tickers)
smoothing_sma_ema(SMA[0], tickers)
smoothing_sma_ema(SMA[1], tickers)
smoothing_sma_ema(SMA[2], tickers)
smoothing_sma_ema(SMA[3], tickers)
smoothing_sma_smoothed(SMA[0], tickers)
smoothing_sma_smoothed(SMA[1], tickers)
smoothing_sma_smoothed(SMA[2], tickers)
smoothing_sma_smoothed(SMA[3], tickers)
#smoothing_sma_ar(SMA[0], tickers)
#smoothing_sma_ar(SMA[1], tickers)
#smoothing_sma_ar(SMA[2], tickers)
#smoothing_sma_ar(SMA[3], tickers)
#display([dfsma[(SMA[0] + '_change_smooth_savgol', 'AAPL')], dfsma[(SMA[0] + '_change_smooth_sma', 'AAPL')],dfsma[('SMA10', 'AAPL')]])

for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(8,6))
    ax[0].plot(dfsma[('Close', t)])
    ax[0].plot(dfsma[('SMA10', t)])
    ax[0].plot(dfsma[('SMA20', t)])
    ax[0].plot(dfsma[('SMA50', t)])
    ax[0].plot(dfsma[('SMA100', t)])
    ax[0].plot(dfsma[('SMA200', t)])
    ax[1].plot(dfsma[(SMA[0] + '_change_smooth_sma', t)], linewidth=1)
    #ax[1].plot(dfsma[(SMA[1] + '_change_smooth_sma', t)], linewidth=1)
    #ax[1].plot(dfsma[(SMA[2] + '_change_smooth_sma', t)], linewidth=1)
    #ax[1].plot(dfsma[(SMA[3] + '_change_smooth_sma', t)], linewidth=1)
    ax[1].plot(dfsma[(SMA[0] + '_change_smooth_savgol', t)], color='red')
    #ax[1].plot(dfsma[(SMA[1] + '_change_smooth_savgol', t)], color='b')
    #ax[1].plot(dfsma[(SMA[2] + '_change_smooth_savgol', t)])
    #ax[1].plot(dfsma[(SMA[3] + '_change_smooth_savgol', t)])
    #ax[1].plot(dfsma[(SMA[0] + '_change_smooth_ema', t)])
    #ax[1].plot(dfsma[(SMA[1] + '_change_smooth_ema', t)])
    #ax[1].plot(dfsma[(SMA[2] + '_change_smooth_ema', t)])
    #ax[1].plot(dfsma[(SMA[3] + '_change_smooth_ema', t)])
    #ax[1].plot(dfsma[(SMA[0] + '_change_smooth_smoothed', t)])
    #ax[1].plot(dfsma[(SMA[1] + '_change_smooth_smoothed', t)])
    #ax[1].plot(dfsma[(SMA[2] + '_change_smooth_smoothed', t)])
    #ax[1].plot(dfsma[(SMA[3] + '_change_smooth_smoothed', t)])
    #ax[1].plot(dfsma[(SMA[0] + '_change_smooth_ar', t)])
    #ax[1].plot(dfsma[(SMA[1] + '_change_smooth_ar', t)])
    #ax[1].plot(dfsma[(SMA[2] + '_change_smooth_ar', t)])
    #ax[1].plot(dfsma[(SMA[3] + '_change_smooth_ar', t)])
    #ax[1].plot(dfsma[(SMA[0] + '_change', t)])
    #ax[1].plot(dfsma[(SMA[1] + '_change', t)])
    #ax[1].plot(dfsma[(SMA[2] + '_change', t)])
    #ax[1].plot(dfsma[(SMA[3] + '_change', t)])
    plt.title(t)
    plt.grid()
    #plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
######TEST Place, fuction smoothing_sma is nu bovenaan; probleem met oneven savgol en nog veel meer, heb deze flink verknoeid

#Werkt!
def smoothing_sma(SMA, tickers):
    for t in tickers:
        savgol = savgol_filter(dfsma[(SMA + '_change', t)].dropna(), len(dfsma[(SMA + '_change', t)].dropna()), 6)
        insert_len =  len(dfsma[(SMA + '_change', t)]) - len(savgol)
        savgol = np.insert(savgol, 0, np.zeros(insert_len) + np.nan)
        savgol = pd.Series(savgol, index=dfsma.index)
        dfsma[(SMA + '_change_smooth', t)] = savgol

#~~~~~~~~~~~~~~~~de code los:
print(len(dfsma))
savgol = savgol_filter(dfsma[('SMA100_change', 'AAPL')].dropna(), len(dfsma[('SMA100_change', 'AAPL')].dropna()), 6)
insert_len =  len(dfsma[('SMA100_change', 'AAPL')]) - len(savgol)
print(insert_len)

#1. To add values at a specific place, use: "np.insert()"  -> To add at the end use "append"
#2. Adding a certain number of "NaN"'s is best to do using: np.zeros(~the len that you want NaN~) + np.nan
#Source:https://stackoverflow.com/questions/19539267/add-nan-to-numpy-array-20-times-without-loop
savgol = np.insert(savgol, 0, np.zeros(insert_len) + np.nan)
print(pd.Series(savgol))

#1. Couldn't just add the "array" gotten from savgol_filter, had to change it to "pd.Series" first!
#2. Added column only had "NaN"'s; had to add the "index=dfsma.index" for it to work!!!
#source: https://stackoverflow.com/questions/12555323/adding-new-column-to-existing-dataframe-in-python-pandas
savgol = pd.Series(savgol, index=dfsma.index)
dfsma[('SMA100_change_smooth', 'AAPL')] = savgol
#~~~~~~~~~~~~~~~~EIND

SMA = 'SMA100'
smoothing_sma(SMA, tickers)
smoothing_sma(SMA, tickers)

#print(dfsma[('SMA100_change_smooth', 'AAPL')])
dfsma.head()
#dfsma[('SMA100_change_smooth', 'AAPL')].plot()
plt.figure()
plt.plot(dfsma.index, dfsma[(SMA + '_change', 'MSFT')])
plt.plot(dfsma.index, dfsma[(SMA + '_change_smooth', 'MSFT')])
plt.show()

1005
100
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
1000    0.003502
1001    0.003530
1002    0.003557
1003    0.003585
1004    0.003613
Length: 1005, dtype: float64


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# STATS - o.a. Skewness and kurtosis

In [7]:
#source -> https://www.pythonforfinance.net/2016/04/04/python-skew-kurtosis/
#this source was purely to check for correct implementation

####check for further correctness: https://stackoverflow.com/questions/45483890/how-to-correctly-use-scipys-skew-and-kurtosis-functions

# mine (sentence above) gives the same result as the longer code (below)
df[('Percent_return', 'ISRG')].hist(bins=100)
plt.show()

dist = np.asarray(df[('Percent_return', 'ISRG')].dropna())
dist = sorted(dist)
fit = stats.norm.pdf(dist, np.mean(dist), np.std(dist))
plt.plot(dist, fit, '-', linewidth=2)
plt.hist(dist, density=True, bins=100)
plt.show()

print(f'Skewness for X is: ', df[('Percent_return', 'ISRG')].skew())
print(f'Kurtosis for X is: ', df[('Percent_return', 'ISRG')].kurt())

Skewness for X is:  -0.12115311385500083
Kurtosis for X is:  3.5790127921031964


# Backtesting!

In [412]:
#####Backtesting implementation
#--------------------------------------Check database ERD!!------------------------------------------
#basics

#!!!!!_____________BELANGRIJK: https://stackoverflow.com/questions/35831496/key-error-when-selecting-columns-in-pandas-dataframe-after-read-csv
#if you need to select multiple columns from dataframe use 2 pairs of square brackets eg.
#   print(dfsma.columns)
#   display(dfsma[[('SMA100', 'AAPL'), ('SMA200', 'AAPL')]])

#how to use if statements in df source: https://datatofish.com/if-condition-in-pandas-dataframe/  <--got more on python!
#also discussed in python finance course (HK)
#kan zoveel IFs, ANDs, en ORs gebruiken als wilt
#Applying lambda to multiple columns: https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe

#Control strat; buy and hold
for ticker in tickers:
    df[('Strat_control', ticker)] = df[('Percent_return', ticker)].apply(lambda x: 1)

#perfect strat; buying and selling to maximize percent return (doesn't work irl)
df_stratperfect = dfsma.copy()
for t in tickers:
    df_stratperfect[('Strat_perfect', t)] = [1 if df_stratperfect.loc[i, ('Percent_return', t)]>=0 else 0 for i in df_stratperfect.index]
df_stratperfect2 = dfsma.copy()
for t in tickers:
        for i in df_stratperfect.index:
            if df_stratperfect2.loc[i, ('Percent_return', t)]>=0:
                df_stratperfect2.loc[i+1, ('Strat_perfect', t)] = 1
            else:
                df_stratperfect2.loc[i+1, ('Strat_perfect', t)] = 0
    
######################################
    
#SMA strat 1; buy if 100 > 200, sell if 100 < 200
df_strat1 = dfsma.copy()
for t in tickers:
    df_strat1[('Strat_SMA_1', t)] = [1 if df_strat1.loc[i, ('SMA100', t)]>df_strat1.loc[i, ('SMA200', t)] else 0 for i in df_strat1.index]
df_strat1 = dfsma.copy()
for t in tickers:
        for i in df_stratperfect.index:
            if df_strat1.loc[i, ('SMA100', t)]>df_strat1.loc[i, ('SMA200', t)]:
                df_strat1.loc[i+1, ('Strat_perfect', t)] = 1
            else:
                df_strat1.loc[i+1, ('Strat_perfect', t)] = 0
    
#SMA strat 2; buy if 20smooth_savgol >= 0, sell if not
df_strat2 = dfsma.copy()
for t in tickers:
    df_strat2[('Strat_SMA_2', t)] = [1 if df_strat2.loc[i, ('SMA20_change_smooth_savgol', t)]>=0 else 0 for i in df_strat2.index]
df_strat22 = dfsma.copy()
for t in tickers:
        for i in df_strat2.index:
            if df_strat22.loc[i, ('SMA20_change_smooth_savgol', t)]>=0:
                df_strat22.loc[i+1, ('Strat_SMA_2', t)] = 1
            else:
                df_strat22.loc[i+1, ('Strat_SMA_2', t)] = 0
                
#SMA strat 3; buy if 20sma > 200sma, sell if not
#omdat ik sma200 gebruik, werkt de strat later dan de rest!
df_strat3 = dfsma.copy()
for t in tickers:
    df_strat3[('Strat_SMA_3', t)] = [1 if df_strat3.loc[i, ('SMA20_change', t)]>df_strat3.loc[i, ('SMA200_change', t)] else 0 for i in df_strat3.index]
df_strat32 = dfsma.copy()
for t in tickers:
        for i in df_strat32.index:
            if df_strat32.loc[i, ('SMA20_change', t)]>df_strat32.loc[i, ('SMA200_change', t)]:
                df_strat32.loc[i+1, ('Strat_SMA_3', t)] = 1
            else:
                df_strat32.loc[i+1, ('Strat_SMA_3', t)] = 0

#Strat 4
df_strat4 = dfsma.copy()
for t in tickers:
    df_strat4[('Strat_SMA_4', t)] = [1 if df_strat4.loc[i, ('SMA50_change', t)]>=0 else 0 for i in df_strat4.index]
df_strat42 = dfsma.copy()
for t in tickers:
        for i in df_strat42.index:
            if df_strat42.loc[i, ('SMA50_change', t)]>=0:
                df_strat42.loc[i+1, ('Strat_SMA_4', t)] = 1
            else:
                df_strat42.loc[i+1, ('Strat_SMA_4', t)] = 0
                
#Strat 5
df_strat5 = dfsma.copy()
for t in tickers:
    df_strat5[('Strat_SMA_5', t)] = [1 if df_strat5.loc[i, ('SMA50_change_smooth_sma', t)]>=0 else 0 for i in df_strat5.index]
df_strat52 = dfsma.copy()
for t in tickers:
        for i in df_strat52.index:
            if df_strat52.loc[i, ('SMA50_change_smooth_sma', t)]>=0:
                df_strat52.loc[i+1, ('Strat_SMA_5', t)] = 1
            else:
                df_strat52.loc[i+1, ('Strat_SMA_5', t)] = 0

#Strat 6
def polyfit_df(tickers):
    dftemp = df.copy()
    for t in tickers:
        result = np.polyfit(dftemp[('Percent_return', t)].dropna().index, dftemp[('Percent_return', t)].dropna(), 50)
        poly_eq = np.poly1d(result)
        dftemp[('Polyfit', t)] = poly_eq(dftemp.index)
        #dftemp[('Polyfit_change', t)] = dftemp[('Polyfit', t)].pct_change()  <--????????
    dftemp = dftemp.reset_index()
    return dftemp                 
df_strat6 = polyfit_df(tickers)
for t in tickers:
    df_strat6[('Strat_polyfit_6', t)] = [1 if df_strat6.loc[i, ('Polyfit', t)]>=0 else 0 for i in df_strat6.index]
df_strat62 = polyfit_df(tickers)
for t in tickers:
        for i in df_strat62.index:
            if df_strat62.loc[i, ('Polyfit', t)]>=0:
                df_strat62.loc[i+1, ('Strat_polyfit_6', t)] = 1
            else:
                df_strat62.loc[i+1, ('Strat_polyfit_6', t)] = 0    
                
#Strat 7
def polyfit_df(tickers):
    dftemp = df.copy()
    for t in tickers:
        dftemp[('SMA', t)] = dftemp[('Close', t)].rolling(50).mean()
        dftemp[('SMA_change', t)] = dftemp[('SMA', t)].pct_change()
        dftemp[('SMA_change', t)] = dftemp[('SMA_change', t)].dropna()
        result = np.polyfit(dftemp[('SMA_change', t)].dropna().index, dftemp[('SMA_change', t)].dropna(), 50)
        poly_eq = np.poly1d(result)
        dftemp[('Polyfit', t)] = poly_eq(dftemp.index)
        #dftemp[('Polyfit_change', t)] = dftemp[('Polyfit', t)].pct_change()  <--????????
    dftemp = dftemp.reset_index()
    return dftemp                
df_strat7= polyfit_df(tickers)
for t in tickers:
    df_strat7[('Strat_polyfit_7', t)] = [1 if df_strat7.loc[i, ('Polyfit', t)]>=0 else 0 for i in df_strat7.index]
df_strat72 = polyfit_df(tickers)
for t in tickers:
        for i in df_strat72.index:
            if df_strat72.loc[i, ('Polyfit', t)]>=0:
                df_strat72.loc[i+1, ('Strat_polyfit_7', t)] = 1
            else:
                df_strat72.loc[i+1, ('Strat_polyfit_7', t)] = 0  

#Different figure sizes within a single subplot:  https://stackoverflow.com/questions/10388462/matplotlib-different-size-subplots
#Need to specify if it is: "height_ratios" or "width_ratios" !: https://stackoverflow.com/questions/10388462/matplotlib-different-size-subplots
#"hspace" and "wspace" is for distance between the subplots
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 1], 'hspace':0.02})
ax[1].plot(df_strat2[('Strat_SMA_2', 'INTC')])
ax[1].plot(df_strat22[('Strat_SMA_2', 'INTC')])
ax[0].plot(df_strat2[('Close', 'INTC')])

#fig, ax = plt.subplots(2, 1, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 1], 'hspace':0.02})
#ax[1].plot(df_stratperfect[('Strat_perfect', 'INTC')])
#ax[0].plot(df_stratperfect[('Close', 'INTC')])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [413]:
###Calculating the return of the strats

#Notes:  1. xxx.iloc[200:] is omdat sommige strats uiterlijk na 200 periods werken, dus nodig voor eerlijke vergelijking!

#Control strat; buy and hold
df[('Percent_return', 'Total')] = 0
for t in tickers:
    df[('Percent_return', 'Total')].iloc[200:] += df[('Percent_return', t)].iloc[200:].cumsum()
    
#Strat perfect; buy if Percent return >= 0, sell if not
df_stratperfect[('Strat_perfect_percent_return', 'Total')] = 0
for t in tickers:
    df_stratperfect[('Strat_perfect_percent_return', t)] = df_stratperfect[('Strat_perfect', t)] * df_stratperfect[('Percent_return', t)]
    df_stratperfect[('Strat_perfect_percent_return', 'Total')].iloc[200:] += df_stratperfect[('Strat_perfect_percent_return', t)].iloc[200:].cumsum()

df_stratperfect2[('Strat_perfect_percent_return', 'Total')] = 0
for t in tickers:
    df_stratperfect2[('Strat_perfect_percent_return', t)] = df_stratperfect2[('Strat_perfect', t)] * df_stratperfect2[('Percent_return', t)]
    df_stratperfect2[('Strat_perfect_percent_return', 'Total')].iloc[200:] += df_stratperfect2[('Strat_perfect_percent_return', t)].iloc[200:].cumsum()
#####################################################    

#SMA strat 2; buy if 20%smooth_savgol >= 0, else sell
df_strat2[('Strat_SMA_2_percent_return', 'Total')] = 0
for t in tickers:
    df_strat2[('Strat_SMA_2_percent_return', t)] = df_strat2[('Strat_SMA_2', t)] * df_strat2[('Percent_return', t)]
    df_strat2[('Strat_SMA_2_percent_return', 'Total')].iloc[200:] += df_strat2[('Strat_SMA_2_percent_return', t)].iloc[200:].cumsum()
df_strat22[('Strat_SMA_2_percent_return', 'Total')] = 0
for t in tickers:
    df_strat22[('Strat_SMA_2_percent_return', t)] = df_strat22[('Strat_SMA_2', t)] * df_strat22[('Percent_return', t)]
    df_strat22[('Strat_SMA_2_percent_return', 'Total')].iloc[200:] += df_strat22[('Strat_SMA_2_percent_return', t)].iloc[200:].cumsum()
   
    
#SMA strat 3; buy if 20%smooth_savgol > 200%smooth_savgol, sell if not
df_strat3[('Strat_SMA_3_percent_return', 'Total')] = 0
for t in tickers:
    df_strat3[('Strat_SMA_3_percent_return', t)] = df_strat3[('Strat_SMA_3', t)] * df_strat3[('Percent_return', t)]
    df_strat3[('Strat_SMA_3_percent_return', 'Total')].iloc[200:] += df_strat3[('Strat_SMA_3_percent_return', t)].iloc[200:].cumsum()
df_strat32[('Strat_SMA_3_percent_return', 'Total')] = 0
for t in tickers:
    df_strat32[('Strat_SMA_3_percent_return', t)] = df_strat32[('Strat_SMA_3', t)] * df_strat32[('Percent_return', t)]
    df_strat32[('Strat_SMA_3_percent_return', 'Total')].iloc[200:] += df_strat32[('Strat_SMA_3_percent_return', t)].iloc[200:].cumsum()
    
#SMA strat 4; buy if 20sma > 200sma, sell if not
df_strat4[('Strat_SMA_4_percent_return', 'Total')] = 0
for t in tickers:
    df_strat4[('Strat_SMA_4_percent_return', t)] = df_strat4[('Strat_SMA_4', t)] * df_strat4[('Percent_return', t)]
    df_strat4[('Strat_SMA_4_percent_return', 'Total')].iloc[200:] += df_strat4[('Strat_SMA_4_percent_return', t)].iloc[200:].cumsum()
df_strat42[('Strat_SMA_4_percent_return', 'Total')] = 0
for t in tickers:
    df_strat42[('Strat_SMA_4_percent_return', t)] = df_strat42[('Strat_SMA_4', t)] * df_strat42[('Percent_return', t)]
    df_strat42[('Strat_SMA_4_percent_return', 'Total')].iloc[200:] += df_strat42[('Strat_SMA_4_percent_return', t)].iloc[200:].cumsum()
 
    
#SMA strat 5; buy if 50%smooth_sma >= 0, sell if not
df_strat5[('Strat_SMA_5_percent_return', 'Total')] = 0
for t in tickers:
    df_strat5[('Strat_SMA_5_percent_return', t)] = df_strat5[('Strat_SMA_5', t)] * df_strat5[('Percent_return', t)]
    df_strat5[('Strat_SMA_5_percent_return', 'Total')].iloc[200:] += df_strat5[('Strat_SMA_5_percent_return', t)].iloc[200:].cumsum()
df_strat52[('Strat_SMA_5_percent_return', 'Total')] = 0
for t in tickers:
    df_strat52[('Strat_SMA_5_percent_return', t)] = df_strat52[('Strat_SMA_5', t)] * df_strat52[('Percent_return', t)]
    df_strat52[('Strat_SMA_5_percent_return', 'Total')].iloc[200:] += df_strat52[('Strat_SMA_5_percent_return', t)].iloc[200:].cumsum()

#SMA strat 6; buy if poly_%return >= 0, sell if not
df_strat6[('Strat_polyfit_6_percent_return', 'Total')] = 0
for t in tickers:
    df_strat6[('Strat_polyfit_6_percent_return', t)] = df_strat6[('Strat_polyfit_6', t)] * df_strat6[('Percent_return', t)]
    df_strat6[('Strat_polyfit_6_percent_return', 'Total')].iloc[200:] += df_strat6[('Strat_polyfit_6_percent_return', t)].iloc[200:].cumsum()
df_strat62[('Strat_polyfit_6_percent_return', 'Total')] = 0
for t in tickers:
    df_strat62[('Strat_polyfit_6_percent_return', t)] = df_strat62[('Strat_polyfit_6', t)] * df_strat62[('Percent_return', t)]
    df_strat62[('Strat_polyfit_6_percent_return', 'Total')].iloc[200:] += df_strat62[('Strat_polyfit_6_percent_return', t)].iloc[200:].cumsum()

#SMA strat 7; buy if 30%poly_sma >= 0, sell if not
df_strat7[('Strat_polyfit_7_percent_return', 'Total')] = 0
for t in tickers:
    df_strat7[('Strat_polyfit_7_percent_return', t)] = df_strat7[('Strat_polyfit_7', t)] * df_strat7[('Percent_return', t)]
    df_strat7[('Strat_polyfit_7_percent_return', 'Total')].iloc[200:] += df_strat7[('Strat_polyfit_7_percent_return', t)].iloc[200:].cumsum()
df_strat72[('Strat_polyfit_7_percent_return', 'Total')] = 0
for t in tickers:
    df_strat72[('Strat_polyfit_7_percent_return', t)] = df_strat72[('Strat_polyfit_7', t)] * df_strat72[('Percent_return', t)]
    df_strat72[('Strat_polyfit_7_percent_return', 'Total')].iloc[200:] += df_strat72[('Strat_polyfit_7_percent_return', t)].iloc[200:].cumsum()

    
df_return = pd.DataFrame()
df_return['Date'] = df[('Date', '')]
df_return['Control_strat'] = df[('Percent_return', 'Total')]
df_return['Strat_perfect'] = df_stratperfect[('Strat_perfect_percent_return', 'Total')]
df_return['Strat_perfect2'] = df_stratperfect2[('Strat_perfect_percent_return', 'Total')]
df_return['Strat_SMA_2'] = df_strat2[('Strat_SMA_2_percent_return', 'Total')]
df_return['Strat_SMA_22'] = df_strat22[('Strat_SMA_2_percent_return', 'Total')]
df_return['Strat_SMA_3'] = df_strat3[('Strat_SMA_3_percent_return', 'Total')]
df_return['Strat_SMA_32'] = df_strat32[('Strat_SMA_3_percent_return', 'Total')]
df_return['Strat_SMA_4'] = df_strat4[('Strat_SMA_4_percent_return', 'Total')]
df_return['Strat_SMA_42'] = df_strat42[('Strat_SMA_4_percent_return', 'Total')]
df_return['Strat_SMA_5'] = df_strat5[('Strat_SMA_5_percent_return', 'Total')]
df_return['Strat_SMA_52'] = df_strat52[('Strat_SMA_5_percent_return', 'Total')]
df_return['Strat_polyfit_6'] = df_strat6[('Strat_polyfit_6_percent_return', 'Total')]
df_return['Strat_polyfit_62'] = df_strat62[('Strat_polyfit_6_percent_return', 'Total')]
df_return['Strat_polyfit_7'] = df_strat7[('Strat_polyfit_7_percent_return', 'Total')]
df_return['Strat_polyfit_72'] = df_strat72[('Strat_polyfit_7_percent_return', 'Total')]
display(df_return)
    
returns2 = df_return[('Strat_SMA_2')].iloc[-1]
returns22 = df_return[('Strat_SMA_22')].iloc[-1]
returns3 = df_return[('Strat_SMA_3')].iloc[-1]
returns32 = df_return[('Strat_SMA_32')].iloc[-1]
returns4 = df_return[('Strat_SMA_4')].iloc[-1]
returns42 = df_return[('Strat_SMA_42')].iloc[-1]
returns5 = df_return[('Strat_SMA_5')].iloc[-1]
returns52 = df_return[('Strat_SMA_52')].iloc[-1]
returns6 = df_return[('Strat_polyfit_6')].iloc[-1]
returns62 = df_return[('Strat_polyfit_62')].iloc[-1]
returns7 = df_return[('Strat_polyfit_7')].iloc[-1]
returns72 = df_return[('Strat_polyfit_72')].iloc[-1]
returnsperfect = df_return[('Strat_perfect')].iloc[-1]
returnsperfect2 = df_return[('Strat_perfect2')].iloc[-1]
returnc = df_return[('Control_strat')].iloc[-1]
print(returns2)
print(returns22)
print('/n')
print(returns3)
print(returns32)
print('/n')
print(returns4)
print(returns42)
print('/n')
print(returns5)
print(returns52)
print('/n')
print(returns6)
print(returns62)
print('/n')
print(returns7)
print(returns72)
print('/n')
print(returnsperfect)
print(returnsperfect2)
print('/n')
print(returnc)

fig, ax = plt.subplots(figsize=(10,5))
ax.plot(df[('Percent_return', 'Total')], linewidth=1, color='r')
ax.plot(df_strat22[('Strat_SMA_2_percent_return', 'Total')], linewidth=1)
ax.plot(df_strat32[('Strat_SMA_3_percent_return', 'Total')], linewidth=1)
ax.plot(df_strat42[('Strat_SMA_4_percent_return', 'Total')], linewidth=1)
ax.plot(df_strat52[('Strat_SMA_5_percent_return', 'Total')], linewidth=1)
ax.plot(df_strat62[('Strat_polyfit_6_percent_return', 'Total')], linewidth=1)
ax.plot(df_strat72[('Strat_polyfit_7_percent_return', 'Total')], linewidth=1)
plt.grid()


,Date,Control_strat,Strat_perfect,Strat_perfect2,Strat_SMA_2,Strat_SMA_22,Strat_SMA_3,Strat_SMA_32,Strat_SMA_4,Strat_SMA_42,Strat_SMA_5,Strat_SMA_52,Strat_polyfit_6,Strat_polyfit_62,Strat_polyfit_7,Strat_polyfit_72
0,2016-01-04,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
1,2016-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2016-01-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2016-01-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2016-01-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,2019-12-24,5.980979,49.513474,1.420812,9.602925,9.593996,13.828513,2.854104,11.774229,2.264906,4.305040,4.333615,10.742902,11.195097,6.431616,6.147749
1002,2019-12-26,6.031707,49.571038,1.468894,9.646103,9.637173,13.879240,2.890381,11.824956,2.315634,4.348748,4.377324,10.796363,11.248558,6.482344,6.198477
1003,2019-12-27,6.116755,49.661326,1.470095,9.731151,9.723378,13.964288,2.975429,11.907811,2.400682,4.434792,4.463368,10.882407,11.331413,6.567392,6.283525
1004,2019-12-30,6.151850,49.753423,1.536700,9.766246,9.758473,14.023427,3.010524,11.942906,2.439059,4.486241,4.514817,10.966848,11.382862,6.602487,6.318620


9.89754842949068
9.895730709923502
/n
14.160684716354307
3.1462170427191323
/n
12.08016452147023
2.5763171997243917
/n
4.617256003336913
4.645831716089839
/n
11.096333461346411
11.51181445841075
/n
6.736131965181408
6.455878029062491
/n
49.89174640719621
1.6617949407772072
/n
6.2891080349983675


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# PRACTISE

In [87]:
#selecting and graphing selected data

#Remember: loc = ['AA', #], iloc = [#, #]
print(df.columns)
#Notice: We are using a MultiIndex! This works a bit different from normal indexes -> the df now has sub-indexes
#Pandas explanantion: https://pandas.pydata.org/pandas-docs/version/0.25.0/user_guide/advanced.html
df2 = df.loc[:, [('Date', ''), ('Close', 'AAPL'), ('Close', 'MSFT'), ('Return', 'AAPL'), ('Return', 'MSFT')]]
print(df2.dtypes)

#Suboptimale manier om dit te doen, beter om het via matplotlib te doen! 
#df2.plot(('Date', ''), ('Close', 'AAPL'), figsize=(8,6))
#kan ook de '.xs' method gebruiken, waardoor je het unstacked: https://www.quantopian.com/posts/plotting-multi-index-pandas-dataframe

#it just works :p -> Geen idee waarom x and y als 'variable' wel werkt maar als 'arg' niet? 
#-> Leren hoe ik het werkt, zodat ik matplotlib beter kan!
x = df2.loc[:, ('Date', '')]
y1 = df2.loc[:, ('Close', 'AAPL')]
y2 = df2.loc[:, ('Return', 'AAPL')]
y3 = df2.loc[:, ('Close', 'MSFT')]
y4 = df2.loc[:, ('Return', 'MSFT')]
fig, axs = plt.subplots(4, figsize=(8,9), sharex=True)
axs[0].plot(x, y1)
axs[0].set_title('AAPL')
axs[1].plot(x, y2)

axs[2].plot(x, y3)
axs[2].set_title('MSFT')
axs[3].plot(x, y4)



MultiIndex([(     'Date',        ''),
            ('Adj Close',    'AAPL'),
            ('Adj Close',    'MSFT'),
            ('Adj Close', 'TKWY.AS'),
            (    'Close',    'AAPL'),
            (    'Close',    'MSFT'),
            (    'Close', 'TKWY.AS'),
            (     'High',    'AAPL'),
            (     'High',    'MSFT'),
            (     'High', 'TKWY.AS'),
            (      'Low',    'AAPL'),
            (      'Low',    'MSFT'),
            (      'Low', 'TKWY.AS'),
            (     'Open',    'AAPL'),
            (     'Open',    'MSFT'),
            (     'Open', 'TKWY.AS'),
            (   'Volume',    'AAPL'),
            (   'Volume',    'MSFT'),
            (   'Volume', 'TKWY.AS'),
            (   'Return',    'AAPL'),
            (   'Return',    'MSFT'),
            (   'Return', 'TKWY.AS')],
           names=['Attributes', 'Symbols'])
Attributes  Symbols
Date                   datetime64[ns]
Close       AAPL              float64
            MSFT       

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [292]:
#Is dit het????  ---> moet testen!, verandert dit aanzienelijk met nieuwe data punten, bijv?
#nog steeds kijken voor autoregression, goed om te kunnen!

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'KODK', 'INTC', 'PANW']
startd = datetime.datetime(2019, 1, 1) 
endd = datetime.datetime(2020, 1, 1)

df = data.DataReader(tickers, 'yahoo', startd, endd).reset_index()
for ticker in tickers:
    df[('Percent_return', ticker)] = df[('Close', ticker)].pct_change()
df = df.dropna()

dfsma = df.copy()
for t in tickers:
    dfsma[('SMA10', t)] = dfsma[('Close', t)].rolling(10).mean()
    dfsma[('SMA20', t)] = dfsma[('Close', t)].rolling(20).mean()
    dfsma[('SMA50', t)] = dfsma[('Close', t)].rolling(50).mean()
    dfsma[('SMA100', t)] = dfsma[('Close', t)].rolling(100).mean()
    dfsma[('SMA200', t)] = dfsma[('Close', t)].rolling(200).mean()
    dfsma[('SMA10_change', t)] = dfsma[('SMA20', t)].pct_change()
    dfsma[('SMA20_change', t)] = dfsma[('SMA20', t)].pct_change()
    dfsma[('SMA50_change', t)] = dfsma[('SMA50', t)].pct_change()
    dfsma[('SMA100_change', t)] = dfsma[('SMA100', t)].pct_change()
    dfsma[('SMA200_change', t)] = dfsma[('SMA200', t)].pct_change()
    
#Source: https://stackoverflow.com/questions/53473666/how-to-apply-best-fit-line-to-time-series-in-python
result = np.polyfit(df.index, df[('Percent_return', 'MSFT')], 50)
poly_eq = np.poly1d(result)
y_hat = poly_eq(df.index)

plt.figure(figsize=(10,5))
plt.plot(df[('Percent_return', 'MSFT')], linewidth=0.5)
plt.plot(df.index, y_hat)
plt.show()
#ex-probleem -> kon niet datetime datums gebruiken! -> zorgt voor een error!  -> https://stackoverflow.com/questions/47379053/typeerror-ufunc-subtract-cannot-use-operands-with-types-dtypem8ns-and-dt
dfsma[('SMA20_change', 'MSFT')] = dfsma[('SMA20_change', 'MSFT')].dropna()
result = np.polyfit(dfsma[('SMA20_change', 'MSFT')].dropna().index, dfsma[('SMA20_change', 'MSFT')].dropna(), 50)
poly_eq = np.poly1d(result)
y_hat = poly_eq(dfsma.index)

plt.figure(figsize=(10,5))
plt.plot(dfsma[('SMA20_change', 'MSFT')], linewidth=0.5)
plt.plot(dfsma.index, y_hat)
#
smoothing_sma_savgol('SMA20', tickers) 
plt.plot(dfsma[('SMA20_change_smooth_savgol', 'MSFT')], color='red')
#
plt.xlim(20)
plt.ylim((-0.02, 0.02))
plt.grid()
plt.show()

#polyfit = 50 geeft perfect fit op Savgol for PANW; 16-20

RemoteDataError: No data fetched using 'YahooDailyReader'

In [373]:
df_strat_ichi = dfichi.copy()

for t in tickers:
    for i in df_strat_ichi.index:
        if df_strat_ichi.loc[i, ('Close', t)]>df_strat_ichi.loc[i, ('Kijun_sen', t)]:
            df_strat_ichi.loc[i+1, ('Strat_ichi', t)] = 1
        if df_strat_ichi.loc[i, ('Tenkan_sen', t)]<df_strat_ichi.loc[i, ('Kijun_sen', t)]:
            df_strat_ichi.loc[i+1, ('Strat_ichi', t)] = 0

display(df_strat_ichi[['Close', 'Kijun_sen', 'Strat_ichi']].iloc[100:])
                
df_strat_ichi[('Strat_ichi_8_percent_return', 'Total')] = 0
for t in tickers:
    df_strat_ichi[('Strat_ichi_8_percent_return', t)] = df_strat_ichi[('Strat_ichi', t)].dropna() * df_strat_ichi[('Percent_return', t)]
    df_strat_ichi[('Strat_ichi_8_percent_return', 'Total')] += df_strat_ichi[('Strat_ichi_8_percent_return', t)].iloc[200:].cumsum()

returnichi = df_strat_ichi[('Strat_ichi_8_percent_return', 'Total')].dropna().iloc[-1]
print(returnichi)

Attributes       Close                                                 \
Symbols            SPY       AAPL        MSFT        PYPL        ISRG   
101         210.240005  25.087500   52.320000   38.090000  212.216660   
102         209.839996  24.965000   53.000000   37.790001  211.570007   
103         210.270004  24.615000   52.849998   38.310001  212.563339   
104         210.910004  24.430000   52.480000   38.560001  213.146667   
105         210.279999  24.480000   51.790001   38.330002  212.279999   
...                ...        ...         ...         ...         ...   
1002        322.940002  72.477501  158.669998  109.750000  598.039978   
1003        322.859985  72.449997  158.960007  109.400002  597.880005   
1004        321.079987  72.879997  157.589996  107.970001  591.320007   
1005        321.859985  73.412498  157.699997  108.169998  591.150024   
1006               NaN        NaN         NaN         NaN         NaN   

Attributes                                            Kijun_sen  ...  \
Symbols            GM   KODK       INTC        PANW         SPY  ...   
101         31.389999  12.74  31.570000  129.860001  206.514999  ...   
102         31.280001  13.11  31.590000  130.460007  206.735001  ...   
103         30.219999  13.45  31.660000  134.470001  206.735001  ...   
104         30.260000  13.90  31.760000  140.729996  206.854996  ...   
105         29.600000  13.85  31.620001  139.639999  206.854996  ...   
...               ...    ...        ...         ...         ...  ...   
1002        36.480000   3.54  59.820000  233.539993  315.040009  ...   
1003        36.560001   3.83  60.080002  233.270004  315.464996  ...   
1004        36.439999   4.16  59.619999  231.429993  315.464996  ...   
1005        36.599998   4.65  59.849998  231.250000  315.464996  ...   
1006              NaN    NaN        NaN         NaN         NaN  ...   

Attributes             Strat_ichi                                          
Symbols           PANW        SPY AAPL MSFT PYPL ISRG   GM KODK INTC PANW  
101         142.764999        0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
102         142.764999        1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0  
103         142.764999        1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0  
104         142.764999        1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0  
105         142.764999        1.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0  0.0  
...                ...        ...  ...  ...  ...  ...  ...  ...  ...  ...  
1002        232.924995        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  
1003        232.924995        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  
1004        232.924995        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  
1005        232.924995        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  
1006               NaN        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  

[906 rows x 27 columns]

2.421633805163305


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [393]:

plt.figure()
plt.plot(df_strat_ichi[('Strat_ichi_8_percent_return', 'SPY')].cumsum())
plt.plot(df_strat_ichi[('Percent_return', 'SPY')].iloc[26:].cumsum())

plt.figure()
plt.plot(df_strat72[('Strat_polyfit_7_percent_return', 'SPY')].cumsum())
plt.plot(df_strat_ichi[('Percent_return', 'SPY')].cumsum())

plt.figure()
plt.plot(df_strat32[('Strat_SMA_3_percent_return', 'SPY')].iloc[200:].cumsum())
plt.plot(df_strat_ichi[('Percent_return', 'SPY')].iloc[200:].cumsum())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …